## 1. Importação de Bibliotecas e Configurações

In [1]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import pickle
import ast
from joblib import load
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    accuracy_score, precision_score, recall_score, 
    f1_score, roc_auc_score, roc_curve
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configurações de plotagem
plt.rcParams['figure.figsize'] = [15, 10]
sns.set_style("whitegrid")

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


## 2. Funções Auxiliares

In [2]:
# Função auxiliar para cálculo do G-Mean
def gmean_score(y_true, y_pred):
    """Calcula o G-Mean (Geometric Mean) para problemas binários"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    return np.sqrt(sensitivity * specificity)

def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """Avalia um modelo treinado em conjuntos de treino e teste"""
    print(f"Avaliando {model_name}...")
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Métricas de treino
    train_metrics = {
        'accuracy': accuracy_score(y_train, y_train_pred),
        'precision': precision_score(y_train, y_train_pred),
        'recall': recall_score(y_train, y_train_pred),
        'f1': f1_score(y_train, y_train_pred),
        'gmean': gmean_score(y_train, y_train_pred)
    }
    
    # Métricas de teste
    test_metrics = {
        'accuracy': accuracy_score(y_test, y_test_pred),
        'precision': precision_score(y_test, y_test_pred),
        'recall': recall_score(y_test, y_test_pred),
        'f1': f1_score(y_test, y_test_pred),
        'gmean': gmean_score(y_test, y_test_pred)
    }
    
    # AUC se o modelo suporta predict_proba
    try:
        y_test_proba = model.predict_proba(X_test)[:, 1]
        test_metrics['auc_roc'] = roc_auc_score(y_test, y_test_proba)
        train_y_proba = model.predict_proba(X_train)[:, 1]
        train_metrics['auc_roc'] = roc_auc_score(y_train, train_y_proba)
    except:
        test_metrics['auc_roc'] = None
        train_metrics['auc_roc'] = None
    
    return train_metrics, test_metrics, y_test_pred

print("✅ Funções auxiliares definidas!")

✅ Funções auxiliares definidas!


## 3. Carregamento dos Dados de Teste

In [4]:
# Carregamento dos dados de teste
print("📥 Carregando dados de teste...")

try:
    # Carregar datasets pré-processados
    train_data = pd.read_csv('dataset_sepsis_prepared.csv')
    test_data = pd.read_csv('dataset_sepsis_test_prepared.csv')
    
    print(f"Dataset de treino: {train_data.shape}")
    print(f"Dataset de teste: {test_data.shape}")
    
    # Separar features e target
    X_train = train_data.drop('SepsisLabel', axis=1)
    y_train = train_data['SepsisLabel']
    X_test = test_data.drop('SepsisLabel', axis=1)
    y_test = test_data['SepsisLabel']
    
    # Normalização dos dados
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print("\n📊 Distribuição das classes:")
    print("Treino:", y_train.value_counts().to_dict())
    print("Teste:", y_test.value_counts().to_dict())
    
except FileNotFoundError as e:
    print(f"❌ Erro: Arquivo não encontrado - {e}")
    print("Certifique-se de que os arquivos dataset_sepsis_prepared.csv e dataset_sepsis_test_prepared.csv existem.")

📥 Carregando dados de teste...
Dataset de treino: (853006, 19)
Dataset de teste: (215171, 19)
Dataset de treino: (853006, 19)
Dataset de teste: (215171, 19)

📊 Distribuição das classes:
Treino: {0.0: 831112, 1.0: 21894}
Teste: {0.0: 209675, 1.0: 5496}

📊 Distribuição das classes:
Treino: {0.0: 831112, 1.0: 21894}
Teste: {0.0: 209675, 1.0: 5496}


In [40]:
# Sampling para avaliação mais rápida
_, X_train_sample, _, y_train_sample = train_test_split(
    X_train_scaled, y_train, 
    test_size=0.5, 
    stratify=y_train,
    random_state=10
)

_, X_test_sample, _, y_test_sample = train_test_split(
    X_test_scaled, y_test, 
    test_size=0.5, 
    stratify=y_test,
    random_state=10
)

print(f"\n🎯 Amostras para avaliação:")
print(f"Treino: {X_train_sample.shape[0]:,} amostras")
print(f"Teste: {X_test_sample.shape[0]:,} amostras")


🎯 Amostras para avaliação:
Treino: 426,503 amostras
Teste: 107,586 amostras


## Definir LVQ

In [29]:
# ======================================================================
# 4.2 LEARNING VECTOR QUANTIZATION (LVQ) - IMPLEMENTAÇÃO E BUSCA
# ======================================================================


# Implementação do LVQ como Estimador compatível com scikit-learn
from sklearn.base import BaseEstimator, ClassifierMixin

class LVQClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, prototypes_per_class=1, n_epochs=100, learning_rate=0.01, random_state=None):
        """
        prototypes_per_class : int
            Número de protótipos a serem usados para cada classe.
        n_epochs : int
            Número de épocas (iterações completas sobre os dados de treinamento).
        learning_rate : float
            Taxa de aprendizado utilizada para atualizar os protótipos.
        random_state : int ou None
            Semente para reprodutibilidade.
        """
        self.prototypes_per_class = prototypes_per_class
        self.n_epochs = n_epochs
        self.learning_rate = learning_rate
        self.random_state = random_state

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        if self.random_state is not None:
            np.random.seed(self.random_state)
            
        # Determina as classes únicas
        self.classes_ = np.unique(y)
        n_features = X.shape[1]

        # Inicializa os protótipos
        prototypes = []
        prototype_labels = []
        for c in self.classes_:
            X_c = X[y == c]
            # Se a quantidade de exemplos dessa classe for menor que o número de protótipos desejados
            replace_flag = X_c.shape[0] < self.prototypes_per_class
            indices = np.random.choice(X_c.shape[0], size=self.prototypes_per_class, replace=replace_flag)
            prototypes.append(X_c[indices])
            prototype_labels.extend([c] * self.prototypes_per_class)
            
        self.prototypes_ = np.vstack(prototypes)
        self.prototype_labels_ = np.array(prototype_labels)

        # Treinamento – Algoritmo LVQ
        for epoch in range(self.n_epochs):
            # Embaralha os índices dos exemplos
            indices = np.random.permutation(X.shape[0])
            for i in indices:
                xi = X[i]
                yi = y[i]
                # Calcula as distâncias euclidianas do exemplo xi a todos os protótipos
                distances = np.linalg.norm(self.prototypes_ - xi, axis=1)
                # Encontra o índice do protótipo mais próximo
                j = np.argmin(distances)
                # Atualiza o protótipo:
                if self.prototype_labels_[j] == yi:
                    # Se a classe coincide, aproxima o protótipo do exemplo
                    self.prototypes_[j] += self.learning_rate * (xi - self.prototypes_[j])
                else:
                    # Se as classes forem diferentes, afasta o protótipo do exemplo
                    self.prototypes_[j] -= self.learning_rate * (xi - self.prototypes_[j])
        return self

    def predict(self, X):
        X = np.array(X)
        y_pred = []
        # Para cada exemplo, calcula a distância para cada protótipo
        for xi in X:
            distances = np.linalg.norm(self.prototypes_ - xi, axis=1)
            j = np.argmin(distances)
            y_pred.append(self.prototype_labels_[j])
        return np.array(y_pred)

    def score(self, X, y):
        return accuracy_score(y, self.predict(X))

## 4. Recuperação dos Resultados dos Modelos Treinados

In [36]:
def load_model_results(results_folder='results'):
    """Carrega todos os resultados de avaliação salvos dos notebooks individuais"""
    all_results = {}

    if not os.path.exists(results_folder):
        print(f"❌ Pasta {results_folder} não encontrada!")
        print("Execute os notebooks individuais dos modelos primeiro.")
        return all_results

    # Mapear arquivos para modelos
    result_files = {
        'knn_results.json': 'KNN',
        'lvq_results.json': 'LVQ', 
        'svm_results.json': 'SVM',
        'rf_results.json': 'Random Forest',
        'dt_results.json': 'Decision Tree'
    }

    for filename, model_name in result_files.items():
        filepath = os.path.join(results_folder, filename)
        
        if os.path.exists(filepath):
            try:
                with open(filepath, 'r') as f:
                    result_data = json.load(f)
                all_results[model_name] = result_data
                print(f"✅ {model_name}: Carregado com sucesso")
            except Exception as e:
                print(f"❌ Erro ao carregar {filename}: {e}")
        else:
            print(f"⚠️  Arquivo não encontrado: {filename}")

    return all_results

def load_trained_models(models_folder='models'):
    """Carrega os modelos treinados salvos"""
    trained_models = {}
    
    if not os.path.exists(models_folder):
        print(f"❌ Pasta {models_folder} não encontrada!")
        return trained_models
    
    model_files = {
        'knn_trained.joblib': 'KNN',
        'lvq_trained.joblib': 'LVQ',
        'svm_trained.joblib': 'SVM', 
        'rf_trained.joblib': 'Random Forest',
        'dt_trained.joblib': 'Decision Tree'
    }
    
    for filename, model_name in model_files.items():
        filepath = os.path.join(models_folder, filename)
        
        if os.path.exists(filepath):
            try:
                model = load(filepath)
                trained_models[model_name] = model
                print(f"✅ Modelo {model_name}: Carregado com sucesso")
            except Exception as e:
                print(f"❌ Erro ao carregar modelo {filename}: {e}")
        else:
            print(f"⚠️  Modelo não encontrado: {filename}")
    
    # Carregar scaler se existir
    scaler_path = os.path.join(models_folder, 'scaler.joblib')
    if os.path.exists(scaler_path):
        try:
            trained_models['scaler'] = load(scaler_path)
            print("✅ Scaler: Carregado com sucesso")
        except Exception as e:
            print(f"❌ Erro ao carregar scaler: {e}")
    
    return trained_models

print("🔄 Carregando resultados dos modelos...")
final_results = load_model_results()

print("\n🔄 Carregando modelos treinados...")
trained_models = load_trained_models()

print(f"\n📊 Total de resultados carregados: {len(final_results)}")
print(f"📊 Total de modelos carregados: {len([k for k in trained_models.keys() if k != 'scaler'])}")

# Verificar se todos os modelos foram carregados
expected_models = ['KNN', 'LVQ', 'SVM', 'Decision Tree', 'Random Forest']
missing_results = [model for model in expected_models if model not in final_results]
missing_models = [model for model in expected_models if model not in trained_models]

if missing_results:
    print(f"⚠️ Resultados faltando: {missing_results}")
if missing_models:
    print(f"⚠️ Modelos faltando: {missing_models}")
    
if not missing_results and not missing_models:
    print("\n✅ Todos os modelos e resultados foram carregados com sucesso!")
else:
    print("\n⚠️ Execute os notebooks individuais correspondentes primeiro.")
print(trained_models)


🔄 Carregando resultados dos modelos...
⚠️  Arquivo não encontrado: knn_results.json
⚠️  Arquivo não encontrado: lvq_results.json
⚠️  Arquivo não encontrado: svm_results.json
⚠️  Arquivo não encontrado: rf_results.json
⚠️  Arquivo não encontrado: dt_results.json

🔄 Carregando modelos treinados...
⚠️  Modelo não encontrado: knn_trained.joblib
✅ Modelo LVQ: Carregado com sucesso
✅ Modelo SVM: Carregado com sucesso
⚠️  Modelo não encontrado: rf_trained.joblib
✅ Modelo Decision Tree: Carregado com sucesso
✅ Scaler: Carregado com sucesso

📊 Total de resultados carregados: 0
📊 Total de modelos carregados: 3
⚠️ Resultados faltando: ['KNN', 'LVQ', 'SVM', 'Decision Tree', 'Random Forest']
⚠️ Modelos faltando: ['KNN', 'Random Forest']

⚠️ Execute os notebooks individuais correspondentes primeiro.
{'LVQ': LVQClassifier(learning_rate=np.float64(0.2189698132600466), n_epochs=879,
              prototypes_per_class=78), 'SVM': SVC(C=np.float64(413.1003988473893), gamma=np.float64(18.059431965634634),

In [46]:
lvq = LVQClassifier(learning_rate=np.float64(0.002749027093115263), n_epochs=131,
              prototypes_per_class=5, random_state=42)

_, X_amostra, _, y_amostra = train_test_split(
    X_train_scaled, y_train, 
    test_size=0.1,  
    stratify=y_train,
    random_state=10
)
lvq.fit(X_amostra, y_amostra)  # Treino com dataset completo
trained_models['LVQ'] = lvq

## 5. Avaliação Final dos Modelos no Conjunto de Teste

In [48]:
print("=== AVALIAÇÃO FINAL DOS MODELOS NO CONJUNTO DE TESTE ===")

# Dicionário para armazenar resultados da avaliação atual
current_evaluation = {}

# Avaliar cada modelo carregado
for model_name, model in trained_models.items():
    if model_name == 'scaler':
        continue
        
    print(f"\n--- Avaliando {model_name} ---")
    if model_name == 'Decision Tree': # alterar max_features
        model.max_features = 'sqrt'
    try:
        # Avaliação no conjunto de teste amostrado
        train_metrics, test_metrics, y_pred = evaluate_model(
            model, X_train_sample, X_test_sample, y_train_sample, y_test_sample, model_name
        )
        
        # Armazenar resultados
        current_evaluation[model_name] = {
            'train_metrics': train_metrics,
            'test_metrics': test_metrics,
            'predictions': y_pred
        }
        
        # Exibir métricas principais
        print(f"Acurácia Treino: {train_metrics['accuracy']:.4f}")
        print(f"F1-Score Treino: {train_metrics['f1']:.4f}")
        print(f"F1-Score Teste: {test_metrics['f1']:.4f}")
        print(f"Recall Teste: {test_metrics['recall']:.4f}")
        print(f"Precisão Teste: {test_metrics['precision']:.4f}")
        print(f"G-Mean Teste: {test_metrics['gmean']:.4f}")
        if test_metrics['auc_roc']:
            print(f"AUC-ROC Teste: {test_metrics['auc_roc']:.4f}")
            
    except Exception as e:
        print(f"❌ Erro ao avaliar {model_name}: {e}")
        continue

print("\n✅ Avaliação de todos os modelos concluída!")

=== AVALIAÇÃO FINAL DOS MODELOS NO CONJUNTO DE TESTE ===

--- Avaliando LVQ ---
Avaliando LVQ...
Acurácia Treino: 0.9742
F1-Score Treino: 0.0036
F1-Score Teste: 0.0007
Recall Teste: 0.0004
Precisão Teste: 0.5000
G-Mean Teste: 0.0191

--- Avaliando SVM ---
Avaliando SVM...
Acurácia Treino: 0.9742
F1-Score Treino: 0.0036
F1-Score Teste: 0.0007
Recall Teste: 0.0004
Precisão Teste: 0.5000
G-Mean Teste: 0.0191

--- Avaliando SVM ---
Avaliando SVM...
Acurácia Treino: 0.9492
F1-Score Treino: 0.0178
F1-Score Teste: 0.0000
Recall Teste: 0.0000
Precisão Teste: 0.0000
G-Mean Teste: 0.0000

--- Avaliando Decision Tree ---
Avaliando Decision Tree...
Acurácia Treino: 0.9492
F1-Score Treino: 0.0178
F1-Score Teste: 0.0000
Recall Teste: 0.0000
Precisão Teste: 0.0000
G-Mean Teste: 0.0000

--- Avaliando Decision Tree ---
Avaliando Decision Tree...
Acurácia Treino: 0.9843
F1-Score Treino: 0.6400
F1-Score Teste: 0.1907
Recall Teste: 0.1878
Precisão Teste: 0.1937
G-Mean Teste: 0.4289
AUC-ROC Teste: 0.6227



## 6. Consolidação de Resultados

In [ ]:
# Combinar resultados carregados com avaliação atual
print("=== CONSOLIDANDO RESULTADOS ===")

consolidated_results = {}

for model_name in expected_models:
    consolidated_results[model_name] = {
        'cv_score': final_results.get(model_name, {}).get('cv_score', 0.0),
        'best_params': final_results.get(model_name, {}).get('best_params', {}),
        'train_metrics': current_evaluation.get(model_name, {}).get('train_metrics', {}),
        'test_metrics': current_evaluation.get(model_name, {}).get('test_metrics', {}),
        'predictions': current_evaluation.get(model_name, {}).get('predictions', [])
    }

# Criar DataFrame com métricas para comparação
metrics_data = []
for model_name, results in consolidated_results.items():
    if not results['test_metrics']:  # Pular se não há métricas de teste
        continue
        
    train_metrics = results.get('train_metrics', {})
    test_metrics = results.get('test_metrics', {})
    
    metrics_data.append({
        'Modelo': model_name,
        'F1_CV': results.get('cv_score', 0.0),
        'F1_Treino': train_metrics.get('f1', 0.0),
        'F1_Teste': test_metrics.get('f1', 0.0),
        'Precisão_Teste': test_metrics.get('precision', 0.0),
        'Recall_Teste': test_metrics.get('recall', 0.0),
        'G-Mean_Teste': test_metrics.get('gmean', 0.0),
        'AUC_ROC_Teste': test_metrics.get('auc_roc', 0.0) if test_metrics.get('auc_roc') else 0.0
    })

metrics_df = pd.DataFrame(metrics_data)

print("=== RESUMO COMPARATIVO DOS MODELOS ===")
print(metrics_df.round(4))

# Salvar resultados consolidados
metrics_df.to_csv('main_evaluation_results.csv', index=False)
print("\n💾 Resultados salvos em 'main_evaluation_results.csv'")

## 7. Visualizações Comparativas

In [ ]:
# Gráficos de comparação
fig, axes = plt.subplots(2, 2, figsize=(20, 15))

# Gráfico 1: Comparação de F1-Score
ax1 = axes[0, 0]
x_pos = np.arange(len(metrics_df))
ax1.bar(x_pos - 0.2, metrics_df['F1_CV'], 0.4, label='F1-Score CV', alpha=0.8, color='skyblue')
ax1.bar(x_pos + 0.2, metrics_df['F1_Teste'], 0.4, label='F1-Score Teste', alpha=0.8, color='orange')
ax1.set_xlabel('Modelos')
ax1.set_ylabel('F1-Score')
ax1.set_title('Comparação F1-Score: CV vs Teste')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(metrics_df['Modelo'], rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Comparação de múltiplas métricas
ax2 = axes[0, 1]
metrics_to_plot = ['F1_Teste', 'Precisão_Teste', 'Recall_Teste', 'G-Mean_Teste']
x = np.arange(len(metrics_df))
width = 0.2

for i, metric in enumerate(metrics_to_plot):
    ax2.bar(x + i*width, metrics_df[metric], width, label=metric.replace('_', ' '), alpha=0.8)

ax2.set_xlabel('Modelos')
ax2.set_ylabel('Score')
ax2.set_title('Comparação de Múltiplas Métricas')
ax2.set_xticks(x + width * 1.5)
ax2.set_xticklabels(metrics_df['Modelo'], rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gráfico 3: Análise de Overfitting
ax3 = axes[1, 0]
ax3.plot(metrics_df['Modelo'], metrics_df['F1_Treino'], 'ro-', label='F1-Score Treino', alpha=0.7)
ax3.plot(metrics_df['Modelo'], metrics_df['F1_Teste'], 'bo-', label='F1-Score Teste', alpha=0.7)
ax3.set_xlabel('Modelos')
ax3.set_title('Análise de Overfitting: F1-Score Treino vs Teste')
ax3.set_ylabel('F1-Score')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.tick_params(axis='x', rotation=45)

# Gráfico 4: AUC-ROC Comparison (para modelos que suportam)
ax4 = axes[1, 1]
auc_data = metrics_df[metrics_df['AUC_ROC_Teste'] > 0]
if not auc_data.empty:
    ax4.bar(auc_data['Modelo'], auc_data['AUC_ROC_Teste'], alpha=0.8, color='purple')
    ax4.set_xlabel('Modelos')
    ax4.set_ylabel('AUC-ROC')
    ax4.set_title('Comparação AUC-ROC')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(True, alpha=0.3)
else:
    ax4.text(0.5, 0.5, 'Nenhum modelo\ncom AUC-ROC disponível', 
             ha='center', va='center', transform=ax4.transAxes, fontsize=12)
    ax4.set_title('AUC-ROC não disponível')

plt.tight_layout()
plt.savefig('main_evaluation_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Gráficos de comparação gerados!")

## 8. Matrizes de Confusão

In [ ]:
# Plotar matrizes de confusão para todos os modelos
n_models = len([m for m in consolidated_results.keys() if consolidated_results[m]['test_metrics']])
if n_models > 0:
    cols = min(3, n_models)
    rows = (n_models + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
    if n_models == 1:
        axes = [axes]
    elif rows == 1:
        axes = axes.reshape(1, -1)
    
    plot_idx = 0
    for model_name, results in consolidated_results.items():
        if not results['test_metrics']:
            continue
            
        row = plot_idx // cols
        col = plot_idx % cols
        ax = axes[row, col] if rows > 1 else axes[col]
        
        # Calcular matriz de confusão
        y_pred = results['predictions']
        cm = confusion_matrix(y_test_sample, y_pred)
        
        # Plotar matriz de confusão
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
        ax.set_title(f'Matriz de Confusão - {model_name}')
        ax.set_xlabel('Predito')
        ax.set_ylabel('Real')
        
        plot_idx += 1
    
    # Remover subplots extras
    for i in range(plot_idx, rows * cols):
        row = i // cols
        col = i % cols
        ax = axes[row, col] if rows > 1 else axes[col]
        ax.remove()
    
    plt.tight_layout()
    plt.savefig('confusion_matrices.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("📊 Matrizes de confusão geradas!")
else:
    print("⚠️ Nenhum modelo com resultados para plotar matrizes de confusão.")

## 9. Curvas ROC

In [ ]:
# Plotar curvas ROC para modelos que suportam predict_proba
plt.figure(figsize=(12, 8))

models_with_roc = []
for model_name, model in trained_models.items():
    if model_name == 'scaler':
        continue
        
    try:
        if hasattr(model, 'predict_proba'):
            y_proba = model.predict_proba(X_test_sample)[:, 1]
            fpr, tpr, _ = roc_curve(y_test_sample, y_proba)
            auc = roc_auc_score(y_test_sample, y_proba)
            
            plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.3f})', linewidth=2)
            models_with_roc.append(model_name)
    except Exception as e:
        print(f"⚠️ Não foi possível plotar ROC para {model_name}: {e}")

if models_with_roc:
    plt.plot([0, 1], [0, 1], 'k--', label='Linha Aleatória', alpha=0.5)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falsos Positivos')
    plt.ylabel('Taxa de Verdadeiros Positivos')
    plt.title('Curvas ROC - Comparação dos Modelos')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.savefig('roc_curves.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"📊 Curvas ROC geradas para {len(models_with_roc)} modelos!")
else:
    print("⚠️ Nenhum modelo suporta predict_proba para curvas ROC.")

## 10. Análise de Overfitting e Underfitting

In [ ]:
print("=== ANÁLISE DE OVERFITTING VS UNDERFITTING ===")

overfitting_analysis = []

for model_name, results in consolidated_results.items():
    if not results['test_metrics'] or not results['train_metrics']:
        continue
        
    f1_train = results['train_metrics'].get('f1', 0.0)
    f1_test = results['test_metrics'].get('f1', 0.0)
    difference = f1_train - f1_test
    
    if difference > 0.1:
        status = "🔴 POSSÍVEL OVERFITTING"
    elif difference < -0.05:
        status = "🔵 POSSÍVEL UNDERFITTING"  
    else:
        status = "✅ BOM EQUILÍBRIO"
    
    overfitting_analysis.append({
        'Modelo': model_name,
        'F1_Treino': f1_train,
        'F1_Teste': f1_test,
        'Diferença': difference,
        'Status': status
    })
    
    print(f"{model_name:15} | Treino: {f1_train:.4f} | Teste: {f1_test:.4f} | Diff: {difference:+.4f} | {status}")

overfitting_df = pd.DataFrame(overfitting_analysis)
overfitting_df.to_csv('overfitting_analysis.csv', index=False)
print("\n💾 Análise de overfitting salva em 'overfitting_analysis.csv'")

## 11. Identificação do Melhor Modelo

In [ ]:
print("=== IDENTIFICAÇÃO DO MELHOR MODELO ===")

if not metrics_df.empty:
    # Encontrar o melhor modelo baseado no F1-Score de teste
    best_model_idx = metrics_df['F1_Teste'].idxmax()
    best_model_name = metrics_df.loc[best_model_idx, 'Modelo']
    best_model_f1 = metrics_df.loc[best_model_idx, 'F1_Teste']
    
    print(f"\n🏆 MELHOR MODELO: {best_model_name}")
    print(f"F1-Score no teste: {best_model_f1:.4f}")
    
    # Detalhes do melhor modelo
    best_model_results = consolidated_results[best_model_name]
    print(f"\n📊 DETALHES DO MELHOR MODELO ({best_model_name}):")
    print(f"F1-Score CV: {best_model_results.get('cv_score', 0.0):.4f}")
    print(f"Acurácia Teste: {best_model_results['test_metrics'].get('accuracy', 0.0):.4f}")
    print(f"Precisão Teste: {best_model_results['test_metrics'].get('precision', 0.0):.4f}")
    print(f"Recall Teste: {best_model_results['test_metrics'].get('recall', 0.0):.4f}")
    print(f"G-Mean Teste: {best_model_results['test_metrics'].get('gmean', 0.0):.4f}")
    
    if best_model_results['test_metrics'].get('auc_roc'):
        print(f"AUC-ROC Teste: {best_model_results['test_metrics']['auc_roc']:.4f}")
    
    # Melhores hiperparâmetros
    best_params = best_model_results.get('best_params', {})
    if best_params:
        print(f"\n🎯 MELHORES HIPERPARÂMETROS:")
        for param, value in best_params.items():
            if isinstance(value, float):
                print(f"  {param}: {value:.4f}")
            else:
                print(f"  {param}: {value}")
    
    # Análise qualitativa do desempenho
    performance_quality = "boa" if best_model_f1 > 0.7 else "moderada" if best_model_f1 > 0.3 else "baixa"
    print(f"\n📈 AVALIAÇÃO: F1-Score de {best_model_f1:.4f} indica {performance_quality} capacidade de predição")
    
else:
    print("⚠️ Não há dados suficientes para identificar o melhor modelo.")

## 12. Resumo Final e Conclusões

In [ ]:
print("=== RESUMO FINAL DAS MÉTRICAS ===")

if not metrics_df.empty:
    print(f"{'Modelo':<15} {'F1':<8} {'Acurácia':<10} {'Precisão':<10} {'Recall':<8} {'G-Mean':<8}")
    print("-" * 70)
    
    for _, row in metrics_df.iterrows():
        print(f"{row['Modelo']:<15} "
              f"{row['F1_Teste']:<8.4f} "
              f"{row.get('Acurácia_Teste', 0.0):<10.4f} "
              f"{row['Precisão_Teste']:<10.4f} "
              f"{row['Recall_Teste']:<8.4f} "
              f"{row['G-Mean_Teste']:<8.4f}")

print("\n=== CONCLUSÕES FINAIS ===")
if not metrics_df.empty:
    best_model_name = metrics_df.loc[metrics_df['F1_Teste'].idxmax(), 'Modelo']
    best_model_f1 = metrics_df['F1_Teste'].max()
    
    print(f"• O modelo {best_model_name} apresentou o melhor desempenho em dados não vistos")
    print(f"• F1-Score de {best_model_f1:.4f} indica {'boa' if best_model_f1 > 0.7 else 'moderada' if best_model_f1 > 0.3 else 'baixa'} capacidade de predição")
    print("• A busca de hiperparâmetros com RandomizedSearchCV e validação cruzada estratificada")
    print("  garantiu seleção robusta dos melhores parâmetros")
    print("• Métricas como G-Mean são importantes para datasets desbalanceados como este")
    print(f"• Total de {len(metrics_df)} modelos foram comparados nesta análise")
else:
    print("• Não foi possível realizar comparação completa dos modelos")
    print("• Execute os notebooks individuais dos modelos primeiro")

print("\n📄 ARQUIVOS GERADOS:")
print("• main_evaluation_results.csv - Resultados comparativos")
print("• overfitting_analysis.csv - Análise de overfitting")
print("• main_evaluation_comparison.png - Gráficos comparativos")
print("• confusion_matrices.png - Matrizes de confusão")
print("• roc_curves.png - Curvas ROC (se disponível)")

print("\n✅ ANÁLISE PRINCIPAL CONCLUÍDA!")
print(f"📊 Total de modelos analisados: {len(consolidated_results)}")